# 1.2 Understanding Prompt Caching

**Duration**: 30 minutes

**Learning Objectives**:
- Understand cache mechanics (TTL, cache hits/misses, cache write/read)
- Learn the cost structure of prompt caching
- Calculate break-even points for caching ROI
- Know which models support caching and their token requirements

---

## Setup

In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import json
from datetime import datetime, timedelta

# Verify AWS credentials are loaded (without exposing sensitive values)
# Credentials can come from .env, AWS CLI (~/.aws/credentials), or environment variables
if os.getenv('AWS_ACCESS_KEY_ID') and os.getenv('AWS_SECRET_ACCESS_KEY'):
    print("✅ AWS credentials loaded successfully")
    print(f"✅ Region: {os.getenv('AWS_DEFAULT_REGION', 'us-east-1')}")
else:
    print("✅ Environment loaded (AWS CLI credentials will be used)")
    print("ℹ️  boto3 will automatically use AWS CLI credentials from ~/.aws/credentials")

✅ Environment loaded (AWS CLI credentials will be used)
ℹ️  boto3 will automatically use AWS CLI credentials from ~/.aws/credentials


## What is Prompt Caching?

**Prompt caching** is an Amazon Bedrock feature that stores frequently used portions of prompts (such as context documents, system instructions, tool definitions) to avoid recomputation across multiple API calls.

### Key Concept

**Cache the static, reuse it repeatedly.**

Instead of processing the same 5,000-token document on every request, cache it once and only process the dynamic user query (e.g., 50 tokens) on subsequent requests.

### Benefits

1. **Cost Savings**: Cache reads cost ~90% less than regular input tokens
2. **Latency Reduction**: Skip recomputation of cached content (50-85% faster)
3. **Scale Efficiency**: Benefits compound with high request volumes

---

## Cache Mechanics

### Time-To-Live (TTL)

- **Cache duration**: 5 minutes
- **TTL resets** on each cache hit (extending cache lifetime)
- **After 5 minutes** of no hits, cache expires and must be rewritten

### Cache Hit

Occurs when the prompt prefix matches exactly with cached content.

**Result**:
- Significantly reduced cost (~90% savings on cached portion)
- Reduced latency (skip processing of cached tokens)
- Only the uncached portion (dynamic content) is processed

### Cache Miss

Occurs when prompt prefix differs from cached content.

**Result**:
- Full recomputation required
- Results in cache write (higher cost than regular input)

**Common causes**:
- Content changes (modified document, updated system prompt)
- Cache expiration (5 minutes elapsed with no hits)
- First request (no cache exists yet)

### Cache Write

First occurrence of a cacheable prompt prefix.

**Cost**: Higher than regular input tokens (typically 1.25x - check current pricing)

**Think of it as**: An investment that pays dividends with subsequent cache reads.

### Cache Read

Subsequent uses of cached content (cache hit scenario).

**Cost**: ~90% less than regular input tokens (typically 0.1x)

**Latency**: Significantly faster (skip token processing)

---

## Cache TTL Simulation

Let's simulate how cache TTL works with requests over time.

In [2]:
class CacheSimulator:
    def __init__(self, ttl_minutes=5):
        self.ttl_minutes = ttl_minutes
        self.cache = None
        self.cache_expiry = None
    
    def request(self, timestamp, content_hash):
        """
        Simulate a request with cache checking.
        
        Args:
            timestamp: datetime object for request time
            content_hash: hash of the cacheable content
        
        Returns:
            'cache_write', 'cache_hit', or 'cache_miss'
        """
        # No cache exists yet
        if self.cache is None:
            self.cache = content_hash
            self.cache_expiry = timestamp + timedelta(minutes=self.ttl_minutes)
            return 'cache_write'
        
        # Cache expired
        if timestamp >= self.cache_expiry:
            self.cache = content_hash
            self.cache_expiry = timestamp + timedelta(minutes=self.ttl_minutes)
            return 'cache_miss (expired)'
        
        # Content changed
        if content_hash != self.cache:
            self.cache = content_hash
            self.cache_expiry = timestamp + timedelta(minutes=self.ttl_minutes)
            return 'cache_miss (content changed)'
        
        # Cache hit! Reset TTL
        self.cache_expiry = timestamp + timedelta(minutes=self.ttl_minutes)
        return 'cache_hit'

# Simulate a series of requests
cache = CacheSimulator(ttl_minutes=5)
start_time = datetime.now()

requests = [
    (0, 'content_v1', 'First request with document A'),
    (1, 'content_v1', 'Query 2 (same document)'),
    (2, 'content_v1', 'Query 3 (same document)'),
    (3, 'content_v1', 'Query 4 (same document)'),
    (4, 'content_v1', 'Query 5 (same document)'),
    (6, 'content_v1', 'Query 6 (6 mins later - TTL reset by previous hit)'),
    (12, 'content_v1', 'Query 7 (12 mins - cache expired!)'),
    (13, 'content_v2', 'Query 8 (content changed!)'),
]

print("=" * 80)
print("CACHE TTL SIMULATION (5-minute TTL)")
print("=" * 80)
print(f"\n{'Time':<8} {'Cache Status':<25} {'Description':<40}")
print("-" * 80)

for minutes, content, description in requests:
    timestamp = start_time + timedelta(minutes=minutes)
    result = cache.request(timestamp, content)
    
    # Format with emoji
    if 'hit' in result:
        status_icon = '✅'
    elif 'write' in result:
        status_icon = '📝'
    else:
        status_icon = '❌'
    
    print(f"{minutes}m       {status_icon} {result:<22} {description}")

print("=" * 80)
print("\n💡 Key Observations:")
print("  - First request = cache write (investment)")
print("  - Subsequent requests within 5 min = cache hit (savings!)")
print("  - Each cache hit RESETS the 5-minute TTL")
print("  - After 5 min of no hits = cache expires")
print("  - Content changes = cache miss (new write required)")
print("=" * 80)

CACHE TTL SIMULATION (5-minute TTL)

Time     Cache Status              Description                             
--------------------------------------------------------------------------------
0m       📝 cache_write            First request with document A
1m       ✅ cache_hit              Query 2 (same document)
2m       ✅ cache_hit              Query 3 (same document)
3m       ✅ cache_hit              Query 4 (same document)
4m       ✅ cache_hit              Query 5 (same document)
6m       ✅ cache_hit              Query 6 (6 mins later - TTL reset by previous hit)
12m       ❌ cache_miss (expired)   Query 7 (12 mins - cache expired!)
13m       ❌ cache_miss (content changed) Query 8 (content changed!)

💡 Key Observations:
  - First request = cache write (investment)
  - Subsequent requests within 5 min = cache hit (savings!)
  - Each cache hit RESETS the 5-minute TTL
  - After 5 min of no hits = cache expires
  - Content changes = cache miss (new write required)


---

## Supported Models and Token Requirements

Not all models support caching, and each has different token minimums.

### Current Supported Models (as of Dec 2025)

| Model Family | Min Tokens per Checkpoint | Max Checkpoints | Cacheable Fields |
|--------------|---------------------------|-----------------|------------------|
| Claude Opus 4.1, Sonnet 4.5, 3.7 Sonnet, Opus 4, Sonnet 4 | 1,024 | 4 | system, messages, tools |
| Claude Opus 4.5, Haiku 4.5 | 4,096 | 4 | system, messages, tools |
| Claude 3.5 Haiku | 2,048 | 4 | system, messages, tools |
| Amazon Nova (Premier, Pro, Lite, Micro) | ~1,024 (max 20K total) | 4 | system, messages |

**Important Notes**:
- Token requirements are **minimums per checkpoint** - content must meet this threshold to be cached
- Claude models support caching in `system`, `messages`, and `tools` fields (we'll explore these in Part 2 labs)
- Nova models support caching in `system` and `messages` fields (tools not cached)
- For current pricing details, check [Amazon Bedrock Pricing](https://aws.amazon.com/bedrock/pricing/)
- For caching implementation details, see [Prompt Caching Documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-caching.html)

---

## Cost Structure: Cache Write vs. Cache Read

Let's calculate the economics of caching.

In [3]:
def calculate_caching_cost(cacheable_tokens, dynamic_tokens, num_requests, 
                           input_price=3.0, cache_write_multiplier=1.25, cache_read_multiplier=0.1):
    """
    Calculate cost comparison: with vs. without caching.
    
    Args:
        cacheable_tokens: Tokens that can be cached (static content)
        dynamic_tokens: Tokens that cannot be cached (user queries)
        num_requests: Total number of requests
        input_price: Price per million input tokens (default: $3.00)
        cache_write_multiplier: Cache write cost multiplier (default: 1.25x)
        cache_read_multiplier: Cache read cost multiplier (default: 0.1x)
    
    Returns:
        dict with cost comparison
    """
    total_tokens = cacheable_tokens + dynamic_tokens
    
    # Without caching: All tokens charged at standard input rate
    cost_no_cache = (total_tokens * num_requests / 1_000_000) * input_price
    
    # With caching:
    # - First request: cache write (1.25x) + dynamic tokens (1x)
    # - Subsequent requests: cache read (0.1x) + dynamic tokens (1x)
    first_request_cost = (
        (cacheable_tokens * cache_write_multiplier / 1_000_000) * input_price +
        (dynamic_tokens / 1_000_000) * input_price
    )
    
    subsequent_request_cost = (
        (cacheable_tokens * cache_read_multiplier / 1_000_000) * input_price +
        (dynamic_tokens / 1_000_000) * input_price
    )
    
    cost_with_cache = first_request_cost + (subsequent_request_cost * (num_requests - 1))
    
    savings = cost_no_cache - cost_with_cache
    savings_pct = (savings / cost_no_cache) * 100
    
    return {
        'cost_no_cache': cost_no_cache,
        'cost_with_cache': cost_with_cache,
        'savings': savings,
        'savings_pct': savings_pct,
        'first_request_cost': first_request_cost,
        'subsequent_request_cost': subsequent_request_cost
    }

# Example: Document Q&A chatbot
# - 5,000 token document (cacheable)
# - 100 token user query (dynamic)
result = calculate_caching_cost(
    cacheable_tokens=5000,
    dynamic_tokens=100,
    num_requests=100
)

print("=" * 80)
print("COST COMPARISON: Caching vs. No Caching")
print("=" * 80)
print(f"\n📊 Scenario: 100 requests with 5,000-token document + 100-token query\n")

print(f"Without caching:")
print(f"  - All requests: ${result['cost_no_cache']:.4f}")

print(f"\nWith caching:")
print(f"  - First request (cache write):   ${result['first_request_cost']:.6f}")
print(f"  - Subsequent requests (cache read): ${result['subsequent_request_cost']:.6f} each")
print(f"  - Total cost: ${result['cost_with_cache']:.4f}")

print(f"\n💰 Savings: ${result['savings']:.4f} ({result['savings_pct']:.1f}% reduction)")
print("=" * 80)

COST COMPARISON: Caching vs. No Caching

📊 Scenario: 100 requests with 5,000-token document + 100-token query

Without caching:
  - All requests: $1.5300

With caching:
  - First request (cache write):   $0.019050
  - Subsequent requests (cache read): $0.001800 each
  - Total cost: $0.1973

💰 Savings: $1.3328 (87.1% reduction)


---

## Break-Even Analysis

How many requests do you need before caching pays off?

**Formula**: Cache write cost > regular input cost, but cache reads cost << regular input

**Break-even point**: When savings from cache reads offset the initial cache write investment.

In [4]:
def find_breakeven_point(cacheable_tokens, dynamic_tokens, input_price=3.0):
    """
    Calculate break-even point for prompt caching.
    
    Returns:
        Number of requests needed to break even
    """
    # Cost without caching per request
    cost_per_request_no_cache = ((cacheable_tokens + dynamic_tokens) / 1_000_000) * input_price
    
    # Cost with caching (first request)
    first_request_cost = (
        (cacheable_tokens * 1.25 / 1_000_000) * input_price +
        (dynamic_tokens / 1_000_000) * input_price
    )
    
    # Cost with caching (subsequent requests)
    subsequent_request_cost = (
        (cacheable_tokens * 0.1 / 1_000_000) * input_price +
        (dynamic_tokens / 1_000_000) * input_price
    )
    
    # Find break-even point
    # No cache: N * cost_per_request_no_cache
    # With cache: first_request_cost + (N-1) * subsequent_request_cost
    # Solve: N * cost_per_request_no_cache = first_request_cost + (N-1) * subsequent_request_cost
    
    breakeven = (first_request_cost - subsequent_request_cost) / (cost_per_request_no_cache - subsequent_request_cost)
    
    return {
        'breakeven_requests': breakeven,
        'cost_per_request_no_cache': cost_per_request_no_cache,
        'first_request_cost': first_request_cost,
        'subsequent_request_cost': subsequent_request_cost
    }

# Calculate break-even for different scenarios
scenarios = [
    (5000, 100, "Large document (5K tokens) + short query"),
    (3000, 200, "Medium document (3K tokens) + medium query"),
    (1500, 500, "Small document (1.5K tokens) + long query"),
]

print("=" * 80)
print("BREAK-EVEN ANALYSIS")
print("=" * 80)
print(f"\n{'Scenario':<45} {'Break-Even':<15}")
print("-" * 80)

for cacheable, dynamic, description in scenarios:
    result = find_breakeven_point(cacheable, dynamic)
    breakeven_rounded = int(result['breakeven_requests']) + 1
    
    print(f"{description:<45} {breakeven_rounded} requests")

print("\n" + "=" * 80)
print("💡 Key Insight: Break-even is typically 2-3 requests.")
print("   After that, every request saves ~75-90% on cached tokens!")
print("=" * 80)

BREAK-EVEN ANALYSIS

Scenario                                      Break-Even     
--------------------------------------------------------------------------------
Large document (5K tokens) + short query      2 requests
Medium document (3K tokens) + medium query    2 requests
Small document (1.5K tokens) + long query     2 requests

💡 Key Insight: Break-even is typically 2-3 requests.
   After that, every request saves ~75-90% on cached tokens!


---

## When Caching Makes Economic Sense

Use this checklist to determine if caching is beneficial:

### ✅ Good Use Cases for Caching

1. **Document Q&A**: Users ask multiple questions about the same document
2. **Chatbots with context**: Static company policies + dynamic user queries
3. **Agents with tools**: Tool definitions rarely change, but queries vary
4. **Batch processing**: Same prompt template across many inputs
5. **Few-shot learning**: Static examples + dynamic test cases

**Key criteria**:
- Static content ≥ 1,024 tokens (or model minimum)
- Expected 2+ requests within 5 minutes
- Static content reused frequently

### ❌ Poor Use Cases for Caching

1. **One-off requests**: No reuse = no benefit
2. **Highly dynamic content**: Content changes every request = constant cache misses
3. **Small prompts**: Below minimum token threshold (< 1,024 tokens)
4. **Low request volume**: Break-even never reached
5. **Long gaps between requests**: Cache expires (> 5 minutes)

---

## Summary

In this notebook, you learned the mechanics and economics of prompt caching:

### Key Takeaways

1. ✅ **Cache Mechanics**: 
   - 5-minute TTL (resets on each cache hit)
   - Cache hits save ~90% on cached tokens
   - Cache misses trigger cache writes (1.25x cost investment)

2. ✅ **Cost Structure**: 
   - Cache write: 1.25x regular input (one-time investment)
   - Cache read: 0.1x regular input (90% savings)
   - Break-even: Typically 2-3 requests
   - Example: 100 requests with 5K-token document → **87.1% cost reduction**

3. ✅ **Model Support**: 
   - Claude models: 1,024+ tokens minimum (Opus/Sonnet), 4,096+ (Haiku 4.5)
   - Nova models: ~1,024 tokens minimum (max 20K total)
   - Maximum 4 cache checkpoints per request
   - Cacheable fields: system, messages, tools (we'll explore these in Part 2)

4. ✅ **When to Cache**: 
   - Static content ≥ model minimum token threshold
   - Expected 2+ requests within 5 minutes
   - Frequent reuse of static content (documents, policies, tool definitions)
   
---